In [2]:
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Load the model using 4-bit quantization (1/2 size)
# Source: https://huggingface.co/blog/4bit-transformers-bitsandbytes
quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = quantization_config, device_map='cuda')

tokenizer = AutoTokenizer.from_pretrained(model_id, device_map='cuda')

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))"), '(Request ID: dbe6cb73-8f8d-4d67-8766-dbec40c22365)')

In [3]:
import sys; sys.path.append("../")
from prompt import Prompt

In [4]:
print(Prompt.query_decomposer)

# Question Decomposition Specialist

    ## Background
    - You are an expert at analyzing problems and are good at breaking down difficult problems into simple problems.
    - A person facing the problem {question} is asking you for help. The question is hard to answer directly.

    ## Goal
    Helping the user decompose the question and tell the user at the right time that the problem can be solved.

    ## Constraint
    - Forget all the knowledge you've learned before and decide whether to continue decomposing the question based only on the user's answers.
    - To make it easier for the user to answer, only one simple question is asked at once.
    - You can only decompose the question, do not answer it directly.

    ## Workflow
    1. Analyse the original complex question and formulate a simple question based on that complex question.
    2. Receive the user's answer to the simple question at hand.
    2.1 If the user is unable to answer the current simple question, rephrase a

In [6]:
def decompose_question_step(input : str | list, max_tokens : int = 50):
    """
    Given a multi-hop question, decomposes the question *once* to generate a sub-question, or returns "That's enough" if the LLM believes the question has been fully decomposed.
    To answer multi-hop questions, The ``Llama-3.1-8B-Instruct`` model is deployed with an elicitive Chain-of-Thought prompt from ``prompt.py``, which is created using a template from [LangGPT](https://github.com/langgptai/LangGPT/).

    Args:
        input (str/list): Can either be a multi-hop question of data type ``str``, or an ongoing LLM chat history of type ``list``.
            If ``str``, the input is treated as the initial multi-hop question, E.g., ``"Who was president of the United States in the year that Citibank was founded?"``.
            If ``list``, the input is treated as a subsequent step in the query decomposition.
        max_tokens (int): The maximum number of tokens the LLM is allowed to generate for the sub-question.
    
    Returns:
        chat_history (list): The entire chat history generated from the LLM, which includes both the CoT prompt, the user query, and the LLM's decomposition.
                            This argument can be fed back into the ``decompose_question_step`` function to generate further sub-questions once context has been retrieved and implemented for the initial sub-question.
        sub_question (str): The sub-question extracted from the full ``chat_history``. Context can be retrieved for this question.
    """

    if isinstance(input, str):
        input = [
            {'role':'system','content':Prompt.query_decomposer}, # Question Decomposition Specialist Prompt
            {'role':'user','content':f"Let's break down this complex question: {input}"}
        ]

    chat_history = pipeline(
        input,
        temperature=0,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        truncation = True,
        max_new_tokens=50
    )

    assistant_response = chat_history[0]['generated_text'][-1]
    sub_question = assistant_response['content']

    return chat_history, sub_question

In [7]:
chat_history, sub_question = decompose_question_step("Who was president of the United States in the year that Citibank was founded?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'role': 'system', 'content': '# Question Decomposition Specialist\n\n    ## Background\n    - You are an expert at analyzing problems and are good at breaking down difficult problems into simple problems.\n    - A person facing the problem {question} is asking you for help. The question is hard to answer directly.\n\n    ## Goal\n    Helping the user decompose the question and tell the user at the right time that the problem can be solved.\n\n    ## Constraint\n    - Forget all the knowledge you\'ve learned before and decide whether to continue decomposing the question based only on the user\'s answers.\n    - To make it easier for the user to answer, only one simple question is asked at once.\n    - You can only decompose the question, do not answer it directly.\n\n    ## Workflow\n    1. Analyse the original complex question and formulate a simple question based on that complex question.\n    2. Receive the user\'s answer to the simple question at hand.\n    2.1 If the user is unab

c:\Users\Alex\anaconda3\envs\gpu_tf\lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [12]:
print(sub_question)

"## 1st Simple Question\nTo start, let's ask a simple question to begin breaking down the complex question:\n\nWhat year was Citibank founded?\n\nPlease respond with a year or indicate that you're unsure."

In [16]:
sub_question

"## 1st Simple Question\nTo start, let's ask a simple question to begin breaking down the complex question:\n\nWhat year was Citibank founded?\n\nPlease respond with a year or indicate that you're unsure."

In [70]:
from query.embedding_model import EmbeddingModel

embedding = EmbeddingModel()
sub_question_embedding = embedding.get_embedding(sub_question, input_is_query=True)

# Testing

In [ ]:
assistant_response = response[0]['generated_text'][-1]
sub_question = assistant_response['content']

In [1]:
import wikipedia

In [2]:
wikipedia.search("Citibank")[0]

'Citibank'

In [3]:
import wikipedia
page = wikipedia.page("Citibank", auto_suggest=False, redirect=True, preload=False)
page.content

'Citibank, N.A. ("N. A." stands for "National Association"; stylized as citibank) is the primary U.S. banking subsidiary of Citigroup, a financial services multinational corporation. Citibank was founded in 1812 as City Bank of New York, and later became First National City Bank of New York. The bank has branches in 19 countries. The U.S. branches are concentrated in six metropolitan areas, New York City, Chicago, Los Angeles, San Francisco, Washington, D.C., and Miami.\nAs of 2023, Citibank is the fourth-largest bank in the United States in terms of assets.\n\n\n== History ==\n\n\n=== Founding ===\n\n\n=== 19th century ===\nThe City Bank of New York was founded on June 16, 1812. The first president of the City Bank was the statesman and retired Colonel, Samuel Osgood. After Osgood\'s death in August 1813, William Few became President of the bank, staying until 1817, followed by Peter Stagg (1817–1825), Thomas Smith (1825–1827), Isaac Wright (1827–1832), and Thomas Bloodgood (1832–1843

In [76]:
print(page.content)

Citibank, N.A. ("N. A." stands for "National Association"; stylized as citibank) is the primary U.S. banking subsidiary of Citigroup, a financial services multinational corporation. Citibank was founded in 1812 as City Bank of New York, and later became First National City Bank of New York. The bank has branches in 19 countries. The U.S. branches are concentrated in six metropolitan areas, New York City, Chicago, Los Angeles, San Francisco, Washington, D.C., and Miami.
As of 2023, Citibank is the fourth-largest bank in the United States in terms of assets.


== History ==


=== Founding ===


=== 19th century ===
The City Bank of New York was founded on June 16, 1812. The first president of the City Bank was the statesman and retired Colonel, Samuel Osgood. After Osgood's death in August 1813, William Few became President of the bank, staying until 1817, followed by Peter Stagg (1817–1825), Thomas Smith (1825–1827), Isaac Wright (1827–1832), and Thomas Bloodgood (1832–1843). After the 